In [ ]:
---
comments: true
layout: post
title: DOM
description: Document Object Model and usage in game
author: Alex Van Linge
permalink: /_notebooks/PBL_Blogs/2025-2-23-input-output-DOM.ipynb/
categories: [PBL Input/Output]
---

In this blog I will discuss the usage of DOM (or Document Object Model) in my code and explain what DOM is.

## DOM Basics 

Document as a Tree Structure:

The DOM represents an HTML document as a tree basically, where each node is an object representing a part of the document.
The tree starts with the document object, which is the root of the tree. From there, it branches out to elements, attributes, and text nodes.

Nodes and Elements:

Node - The basic unit of the DOM tree. Everything in the DOM is a node, including elements, text, and attributes.
Element - A type of node that represents an HTML element. For example, `div`, `p`, and `a` are elements.

Accessing/Manipulating the DOM:

The DOM provides methods to access and manipulates elements and their attributes. For example, you can use document.getElementById() to access an element by its ID. You can also create, modify, and remove elements using methods like createElement(), appendChild(), and removeChild().

Event Handling:

The DOM allows you to handle events, such as clicks, key presses, and form submissions. You can add event listeners to elements to execute JavaScript code in response to user actions.

Summary:
The DOM is a powerful interface that allows you to interact with and manipulate web documents. By understanding the DOM, you can create very dynamic and interactive web pages that respond to user actions and update their content in real-time.





## Example 

In [ ]:
const GameControl = {
    intervalID: null,
    localStorageTimeKey: "localTimes",
    currentPass: 0,
    currentLevelIndex: 0,
    levelClasses: [],
    path: '',

    start: function(path) {
        GameEnv.create();
        this.levelClasses = [
            // GameLevelDesert, // Commented out to disable GameLevelDesert
            GameLevelWater
        ];
        this.currentLevelIndex = 0;
        this.path = path;
        this.addExitKeyListener();
        this.loadLevel();
    },
    
    loadLevel: function() {
        if (this.currentLevelIndex >= this.levelClasses.length) {
            this.stopTimer();
            return;
        }
        GameEnv.continueLevel = true;
        GameEnv.gameObjects = [];
        this.currentPass = 0;
        const LevelClass = this.levelClasses[this.currentLevelIndex];
        const levelInstance = new LevelClass(this.path);
        GameEnv.currentLevel = levelInstance;
        this.loadLevelObjects(levelInstance);
    },
    
    loadLevelObjects: function(gameInstance) {
        this.initStatsUI();
        // Instantiate the game objects
        for (let object of gameInstance.objects) {
            if (!object.data) object.data = {};
            const instance = new object.class(object.data);
            GameEnv.gameObjects.push(instance);
            console.log(`Added ${object.class.name} to game objects`);
        }
        // Start the game loop
        this.gameLoop();
        updateCollectiblesRemaining(); // Update collectibles remaining on start
    },

    gameLoop: function() {
        // Base case: leave the game loop 
        if (!GameEnv.continueLevel) {
            this.handleLevelEnd();
            return;
        }
        // Nominal case: update the game objects 
        GameEnv.clear();
        for (let object of GameEnv.gameObjects) {
            object.update();  // Update the game objects
        }
        this.handleLevelStart();
        // Recursively call this function at animation frame rate
        requestAnimationFrame(this.gameLoop.bind(this));
    },

    handleLevelStart: function() {
        // First time message for level 0, delay 10 passes
        if (this.currentLevelIndex === 0 && this.currentPass === 10) {
            alert("Start Level.");
        }
        // Recursion tracker
        this.currentPass++;
    },

    handleLevelEnd: function() {
        // More levels to play 
        if (this.currentLevelIndex < this.levelClasses.length - 1) {
            alert("Level ended.");
        } else { // All levels completed
            alert("Game over. All levels completed.");
        }
        // Tear down the game environment
        for (let index = GameEnv.gameObjects.length - 1; index >= 0; index--) {
            GameEnv.gameObjects[index].destroy();
        }
        // Move to the next level
        this.currentLevelIndex++;
        // Go back to the loadLevel function
        this.loadLevel();
    },
    
    resize: function() {
        // Resize the game environment
        GameEnv.resize();
        // Resize the game objects
        for (let object of GameEnv.gameObjects) {
            object.resize(); // Resize the game objects
        }
    },

    addExitKeyListener: function() {
        document.addEventListener('keydown', (event) => {
            if (event.key === 'Escape') {
                GameEnv.continueLevel = false;
            }
        });
    },

    startTimer: function() {
        if (GameEnv.timerActive) {
            console.warn("TIMER ACTIVE: TRUE, TIMER NOT STARTED");
            return;
        }
        
        this.intervalId = setInterval(() => this.updateTimer(), GameEnv.timerInterval);
        GameEnv.timerActive = true;
    },

    stopTimer: function() {   
        if (!GameEnv.timerActive) return;
        
        this.saveTime(GameEnv.time, GameEnv.coinScore);

        GameEnv.timerActive = false;
        GameEnv.time = 0;
        GameEnv.coinScore = 0;
        this.updateCoinDisplay();
        clearInterval(this.intervalID);
    },

    updateTimer: function() {
        const time = GameEnv.time;

        if (GameEnv.timerActive) {
            const newTime = time + GameEnv.timerInterval;
            GameEnv.time = newTime;
            if (document.getElementById('timeScore')) {
                document.getElementById('timeScore').textContent = (time / 1000).toFixed(2);
            }
            return newTime;
        }
        if (document.getElementById('timeScore')) {
            document.getElementById('timeScore').textContent = (time / 1000).toFixed(2);
        }
    },

    saveTime: function(time, score) {
        if (time == 0) return;
        const userID = GameEnv.userID;
        const oldTable = this.getAllTimes();

        const data = {
            userID: userID,
            time: time,
            score: score
        };

        if (!oldTable) {
            localStorage.setItem(this.localStorageTimeKey, JSON.stringify([data]));
            return;
        }

        oldTable.push(data);

        localStorage.setItem(this.localStorageTimeKey, JSON.stringify(oldTable));
    },

    getAllTimes: function() {
        let timeTable = null;

        try {
            timeTable = localStorage.getItem(this.localStorageTimeKey);
        } catch (e) {
            return e;
        }

        return JSON.parse(timeTable);
    },

    initStatsUI: function() {
        createStatsUI();
        updateCollectiblesRemaining(); // Update collectibles remaining on start
    }
};

// Detect window resize events and call the resize function.
window.addEventListener('resize', GameControl.resize.bind(GameControl));

export default GameControl;

## Explanation

GameControl Object:

The GameControl object manages the game, including starting the game, loading levels, handling the game loop, and managing the game environment.

`start` Method:

The start method initializes the game environment by calling GameEnv.create(). It sets up the levels to be loaded and starts the first level by calling this.loadLevel(). It also adds an event listener for the "Escape" key to stop the game.

`loadLevel` Method:

The loadLevel method checks if there are more levels to load. If not, it stops the timer.
It initializes the game environment for the new level and loads the game objects for the level.
It then starts the game loop by calling this.gameLoop().

`loadLevelObjects` Method:

The loadLevelObjects method initializes the stats UI and creates instances of the game objects for the level. It adds these game objects to the GameEnv.gameObjects array and starts the game loop.

`gameLoop` Method:

The gameLoop method is the main game loop that updates the game objects and handles the game logic. It clears the canvas, updates the game objects, and recursively calls itself using requestAnimationFrame.

`handleLevelStart` Method:

The handleLevelStart method displays a message when the first level starts after a delay of 10 passes.

`handleLevelEnd` Method:

The handleLevelEnd method handles the end of a level. It displays a message and tears down the game environment. It then moves to the next level or ends the game if all levels are completed.

`resize` Method:

The resize method resizes the game environment and the game objects when the window is resized.

`addExitKeyListener` Method:

The addExitKeyListener method adds an event listener for the "Escape" key to stop the game.

`startTimer, stopTimer, updateTimer` Methods:

These methods manage the game timer, including starting, stopping, and updating the timer.

`saveTime, getAllTimes` Methods:

These methods save and retrieve the game times from local storage.

`initStatsUI` Method:

The initStatsUI method initializes the stats UI and updates the collectibles remaining.

Event Listener for Window Resize:

An event listener is added to the window to call the resize method when the window is resized.

### How It Works

Game Initialization:

The game is initialized by calling the start method, which sets up the game environment and loads the first level.

Game Loop:

The gameLoop method is called recursively using requestAnimationFrame to update the game objects and handle the game logic.

Event Handling:

Event listeners are added for the "Escape" key to stop the game and for window resize events to resize the game environment and objects.

DOM Manipulation:

The initStatsUI method creates and appends a stats container to the document body. The updateTimer method updates the text content of the timeScore element to display the game time.

This example demonstrates how the GameControl object manages the game, including initializing the game environment, loading levels, handling the game loop, and managing the game timer. It also shows how event listeners are used to handle user input and how the DOM is manipulated to update the game UI.